In [1]:
import requests
from requests.packages.urllib3.util.retry import Retry
import csv
import pandas as pd
import math
import matplotlib.pyplot as plt
from datetime import date
from dateutil.rrule import rrule, MONTHLY
from tqdm import tqdm



In [2]:
# tmin, tmax, tavg, prcp
data_type = "prcp"

# this is the downloaded file name
download_file = 'prcp.csv'    

# this is the cleaned data destination file for one region and measurement
destination_file = 'new_england_prcp.csv' 

# initializing the start and end date, data is downloaded in full months, inclusive,  YYYY, M, D
start_date = date(1951, 1, 1)
end_date = date(2021, 12, 31)

header = ['Area','Area code','Area name','year','month',data_type, 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]

# There are many regions
region_code = 1

First, download data from NOAA website
https://www1.ncdc.noaa.gov/pub/data/daily-grids/docs/nclimgrid-daily_v1-0-0_readme-ftp.txt

In [5]:
with requests.Session() as my_request:
    retries = Retry(total=10)
    
    
    with open(f"{download_file}", "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(header)
    
    # Iterate through months and download data
    for d in tqdm(rrule(MONTHLY, dtstart=start_date, until=end_date)):

        year = d.strftime("%Y")
        month = d.strftime("%m")
        base_url = "https://www1.ncdc.noaa.gov/pub/data/daily-grids/v1-0-0/averages/"
        url_details = f"{year}/{data_type}-{year}{month}-hc1-scaled.csv"
        url = base_url + url_details
        
        request = my_request.get(url)
        data = request.text
        

        with open(download_file, "a") as f:
            f.write(data)

852it [09:26,  1.50it/s]


Now, divide the data into seperate files per region, remove the null values for non existent days

Region Codes: 
"California Region = 18 
"Pacific Northwest = 17 
"New England Region = 1 
"Rio Grande Region = 13


In [8]:
data = pd.read_csv(download_file)

# select one of the many regions in the data set
region = data.loc[data['Area code'] == region_code]   

# Non existent days have -999.99 in dataset and need to be removed
region = region.replace(-999.99,None)   

#Only select the measurement data
df_only_numbers = region.iloc[:,6: ]

df_only_numbers

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,0.33,0.00,0.22,2.43,1.05,0.85,3.15,10.59,0.39,0.00,...,0.64,0.00,13.60,8.94,0.03,0.00,0.08,5.96,0.96,0.32
18,20.91,8.73,0.17,4.35,0.53,0.00,3.65,30.83,0.49,0.00,...,15.19,6.23,2.31,0.66,0.54,1.71,2.13,None,None,None
36,0.22,5.57,0.37,4.09,3.94,0.00,0.45,2.41,0.03,0.04,...,2.86,1.80,4.57,3.80,0.03,0.00,0.00,0.23,6.23,19.91
54,6.07,2.64,25.23,12.20,0.00,0.04,1.19,0.66,2.21,1.63,...,0.01,13.46,2.02,1.85,17.75,1.00,0.00,0.65,1.95,None
72,0.00,0.00,0.00,0.01,0.00,0.00,0.31,0.44,0.37,1.19,...,2.61,2.39,16.86,4.50,0.07,0.04,17.79,10.21,3.98,0.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15246,0.01,9.18,1.80,0.00,3.46,4.74,0.07,2.10,0.34,0.85,...,0.94,11.48,14.28,0.53,0.00,0.02,0.28,1.92,1.39,3.62
15264,0.05,38.76,3.52,1.71,0.00,3.72,7.12,0.08,9.20,21.52,...,0.42,1.54,4.84,17.11,14.01,8.28,1.13,1.9,1.61,None
15282,1.47,0.79,1.04,7.22,8.38,0.16,0.00,0.00,0.01,0.00,...,1.75,0.65,0.06,3.17,9.08,12.79,1.30,0.0,1.54,38.43
15300,7.62,0.07,0.10,0.14,0.00,0.00,0.00,0.00,0.00,0.79,...,6.27,3.27,0.05,0.00,0.46,8.52,0.48,0.06,0.01,None


In [9]:
# Convert it to a time series
df_only_numbers = df_only_numbers.stack()
df_only_numbers

0      1     0.33
       2      0.0
       3     0.22
       4     2.43
       5     1.05
             ... 
15318  27    0.08
       28     0.8
       29    1.97
       30    0.49
       31    0.59
Length: 25933, dtype: object

In [10]:
df_only_numbers = df_only_numbers.dropna()
df_only_numbers

0      1     0.33
       2      0.0
       3     0.22
       4     2.43
       5     1.05
             ... 
15318  27    0.08
       28     0.8
       29    1.97
       30    0.49
       31    0.59
Length: 25933, dtype: object

In [11]:
#add appropriate date index
df_only_numbers.index = pd.date_range(start=start_date, end=end_date, freq="D")
df_only_numbers

1951-01-01    0.33
1951-01-02     0.0
1951-01-03    0.22
1951-01-04    2.43
1951-01-05    1.05
              ... 
2021-12-27    0.08
2021-12-28     0.8
2021-12-29    1.97
2021-12-30    0.49
2021-12-31    0.59
Freq: D, Length: 25933, dtype: object

In [16]:
#Write to csv
my_header = ['measurement']
df_only_numbers.to_csv(destination_file, mode='a', header= my_header, index_label="date")
